In [1]:
import argparse, os
import cv2
import numpy as np
import imageio
import time

In [2]:
def readPicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)#cv2.IMREAD_UNCHANGED is important that the output is (x,y,ChannelRGB)
    return(img)

In [3]:
def savePic(picture,fileName,extention,outPath):
    outPath = outPath+fileName+'.'+extention # combines the path with the name and extention of the file
    print(outPath)
    try:
        #imageio.imwrite(outPath,picture,format=extention)# old way
        cv2.imwrite(outPath,picture)#saves Pictures 
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(') #writes an error
        print('--------------------')

In [4]:
def YUVtoRGB(img):
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2BGR, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    #pictureYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV, cv2.IMREAD_UNCHANGED)
    return pictureRGB

In [6]:
#toDo Add parser with args
###### Imput section
print('Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png')
print('FILENAME MUST HAVE the format: 23-y_pred !')
path = input('Path to pictures who should be converted defaut: ./yuvPic/: ') or './yuvPic/'
inputextention = input('What fileextention do the to read pictures have? [default: png]') or 'png'
outputextention = input('Please type outputextention[default: hdr]: ') or 'hdr'
outputpath = input('Where to write the stiched pictures to? [default: ./hdrOut/]: ') or './hdrOut/'
namePic = input('What should be the name of the stiched pictures? [default: predictedPic]') or 'predictedPic'
wantYUV = input('Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]') or 'no'
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention

Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png
FILENAME MUST HAVE the format: 23-y_pred !
Path to pictures who should be converted defaut: ./yuvPic/: /Users/littledragon/Documents/BA 13022020/Daten/resultate/JSI-GAN_x2_exp1/
What fileextention do the to read pictures have? [default: png]
Please type outputextention[default: hdr]: 
Where to write the stiched pictures to? [default: ./hdrOut/]: /Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/
What should be the name of the stiched pictures? [default: pred]
Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]


In [7]:
#Working Version 10022021
start_time = time.time() #start Timer
#TO DO Parser
#desc ='yuv to RGB'
#parser = argparse.ArgumentParser(description=desc)
#parser.add_argument('--yuv', type=str, default='./', help='path to Folder of yuv images')
#print(parser.parse_args())
#what, b = parser.parse_known_args()

#if what.yuv == './' :
 #   print('yes')
start_time = time.time()
i = 0
print(aOp)
while (i <= aOp-1 ): # read y
    print(i)
    try:
        if ((str(os.listdir(path)[i]).split('-')[1]).split('_')[0]) == 'y': # only searching for y picitures
            name = os.listdir(path)[i] #finding the Name
            print(name +' should be the Y')
            picpath = path + name #combining Name and path
            
            #####Y
            picy = readPicture(picpath) #reads a pic y to find the x,y axes should be the same for all pictures
            yuvPic = np.zeros((int(picy.shape[0]),int(picy.shape[1]),3)) # generates the x and y achses and channels of picture
            yuvPic[:,:,0] = picy # packs the Y in pos 1
            num = (str(os.listdir(path)[i]).split('-')[0])#.split('_')[0]
            newPic = path + num + '-u_'+name.split('_')[1]
            
            #####U
            picu = readPicture(newPic)#reads a picture with Ending U
            yuvPic[:,:,1] = picu # packs the u in pos 0
            newPic = path + num + '-v_'+name.split('_')[1] #Generates the Name for v
            
            #####V
            picv = readPicture(newPic)#reads the new picture with ending V
            yuvPic[:,:,2] = picv # packs the u in pos 2
            #Right for the Testdata ist: 0y2u1v <<<<<<<<<<<<<

            if(wantYUV != 'y'):
                
                yuvPic_hdr = YUVtoRGB((yuvPic/((2**10)-1)).astype(np.float32)) #changes the uint16 into float32 and from YUV to RGB
                savePic(((yuvPic_hdr).astype(np.float32)),(str(i)+namePic),outputextention,outputpath) #saves final RGB pic
                
                helpPath = outputpath+'rgb/'
                if not os.path.exists(helpPath):
                    os.mkdir(helpPath)
                rgbPNG= np.clip((yuvPic_hdr*((2**16)-1)), 0, ((2**16)-1)).astype(np.uint16)
                number = int(os.listdir(path)[i].split('-')[0])
                savePic(rgbPNG,(str(number).zfill(6)),'png',helpPath)
                
            if(wantYUV == 'y'):
                yuvPic = YUVtoRGB(yuvPic)
                savePic(yuvPic,(str(i)+namePic),outputextention,outputpath)#saves final YUV pic
    except:
        print('')
    i = i + 1

print("--- %s seconds ---" % (time.time() - start_time))
print(str((time.time() - start_time)/60))
print('------------------------- Done --------------------')

1317
0
1
2
3
194-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/3predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000194.png
4
5
6
235-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/6predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000235.png
7
8
9
10
11
12
202-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/12predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000202.png
13
62-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/13predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000062.png
14
15
16
321-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCom

/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000292.png
118
119
26-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/119predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000026.png
120
118-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/120predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000118.png
121
407-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/121predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000407.png
122
365-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/122predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000365.png
123
124
352-y_pred

/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000023.png
223
224
225
360-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/225predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000360.png
226
227
228
221-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/228predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000221.png
229
230
231
232
233
216-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/233predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000216.png
234
76-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/234predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/0

326
327
328
329
281-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/329predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000281.png
330
395-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/330predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000395.png
331
332
333
334
335
336
83-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/336predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000083.png
337
338
339
142-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/339predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000142.png
340
175-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022

433
332-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/433predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000332.png
434
435
436
437
71-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/437predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000071.png
438
439
46-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/439predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000046.png
440
178-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/440predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000178.png
441
442
305-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicComp

...

862
417-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/862predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000417.png
863
864
865
866
195-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/866predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000195.png
867
868
869
176-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/869predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000176.png
870
871
48-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/871predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000048.png
872
873
874
875
876
141-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022

/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000306.png
974
975
976
331-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/976predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000331.png
977
978
979
72-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/979predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000072.png
980
981
212-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/981predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000212.png
982
983
984
985
986
987
225-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/987predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedH

1078
1079
1080
1081
1082
1083
171-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1083predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000171.png
1084
146-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1084predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000146.png
1085
78-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1085predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000078.png
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
113-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1097predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000113.png
1098
1099
1100
1101
1102


1191
1192
236-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1192predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000236.png
1193
1194
1195
1196
315-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1196predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000315.png
1197
1198
168-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1198predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000168.png
1199
1200
56-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1200predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000056.png
1201
1202
61-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13

1286
1287
1288
1289
1290
1291
259-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1291predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000259.png
1292
1293
418-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1293predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000418.png
1294
107-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1294predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000107.png
1295
39-y_pred.png should be the Y
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/1295predictedPic.hdr
/Users/littledragon/Documents/BA 13022020/programme/PicCompare/generatedHDR/rgb/000039.png
1296
1297
1298
1299
1300
1301
130-y_pred.png should be the Y
/Users/littled

In [ ]:
#yuvPic

In [ ]:
#rgbPic = YUVtoRGB(())

In [ ]:
#os.listdir(path)[i].split('-')[0]